# KNSB ProPEP Automation

## Setting up the environment

I'll be using a powerful tool known as [PyProPEP](https://github.com/jonnydyer/pypropep/tree/master) for this task. It's a Python interface to [CProPEP](https://rocketworkbench.sourceforge.net/) (*an improvement on ForTran ProPEP*). It calculates everything accurately and efficiently. Also ensure you've set up Jupyter on your machine. First, I'll install the libraries:

In [1]:
# This script updates or installs the necessary Python packages
%pip install -r requirements.txt

  Using cached pypropep-0.1.4.tar.gz (346 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached attrdict-2.0.1-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl (182 kB)
Using cached attrdict-2.0.1-py2.py3-none-any.whl (9.9 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
Failed to build pypropep
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pypropep (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-313
      copying cpropep_build.py -> build\lib.win-amd64-cpython-313
      creating build\lib.win-amd64-cpython-313\pypropep
      copying pypropep\equilibrium.py -> build\lib.win-amd64-cpython-313\pypropep
      copying pypropep\error.py -> build\lib.win-amd64-cpython-313\pypropep
      copying pypropep\performance.py -> build\lib.win-amd64-cpython-313\pypropep
      copying pypropep\propellant.py -> build\lib.win-amd64-cpython-313\pypropep
      copying pypropep\__init__.py -> build\lib.win-amd64-cpython-313\pypropep
      creating build\lib.win-amd64-cpython-313\pypropep\cpropep
      copying pypropep\cpropep\__init__.py -> build\lib.win-amd64-cpython-313\pypropep\cpropep
      creating build\lib.win-amd64-cpython-

## Now the fun part...

I'll first start by importing the various libraries we need for this task. 


In [ ]:
import numpy as np
import pandas as pd
import pypropep as ppp
import matplotlib.pyplot as plt

Then I'll define my global variables that will be used throughout the code. For this we'll need the **mechanism** which points to a yaml file that contains the details of the Grains in question and to define some lists like the **flame temp.** and **specific impulse** that will contain the calculated values from our script

We'll then parse the data in the CSV file and get the masses of $KNO_3$ and $C_6H_{14}O_6$ and store them as a dataframe.

In [ ]:
df = pd.read_csv("./csv/base_propep_simulations.csv")
df.head()

We'll start by calculating the propellant desnities, $\rho_p$ and adding them to the csv file. We'll be using the following formula:

$$
\rho_p = \frac{1}{\frac{f_0}{\rho_0}+\frac{f_1}{\rho_1}}
$$

where $f_n$ is the **mass fraction** of the propellant component and $\rho_n$ is the respective density.

In [ ]:
density_kno3 = 2.109
density_sorb = 1.489
densities = []

for _, row in df.iterrows():
    mass_sorb = row["Mass Sorbitol (g)"]
    mass_kno3 = row["Mass KNO3 (g)"]

    f_kno3 = row["KNO3 (%)"] / 100
    f_sorb = row["Sorbitol (%)"] / 100
    
    ratio_kno3 = f_kno3 / density_kno3
    ratio_sorb = f_sorb / density_sorb
    
    density_knsb = (1 / (ratio_kno3 + ratio_sorb)) / 27.68
    densities.append(density_knsb)

df['Density'] = densities
df.head()

Now that's sorted, we can calculate the following data as per the csv file:

#### 1. **Chamber $c_p/c_v$** ($k$ or $\gamma$)

This is the **specific heat ratio** and it is used to determine the chamber pressure. It's given by:

$$
k = \frac{1}{1-\frac{R}{\frac{X}{1-X}C_s+C_p}}
$$

#### 2. **Chamber velocity** ($C^*$)

This is essential in determining the speed at which the propellant will burn at. It's given by:

$$
C^*=\sqrt{\frac{R\:T_o}{k\left(\frac{2}{k+1}\right)^{\frac{k+1}{k-1}}}}
$$


#### 3. **Specific impulse** ($I_{sp}$)

This is the crucial factor that determines the thrust of our SRM. It is given by:

$$
I_{sp}=\frac{I_t}{w_p}
$$

where $w_p$ is the **weight of the propellant** $I_t$ is the **total impulse** that's given by:

$$
I_t=F\cdot {t_b}
$$

where $F$ is the **thrust** and $t_b$ is the **burn time**. For 1 grain the burn time is about 1.5 seconds.

In [ ]:
chamber_pressure = 4.26e+06

chamber_temps = []
gammas: list[float] = []
mol_weights = []
c_stars = []
isps = []

for index, row in df.iterrows():
    mass_kno3 = row['Mass KNO3 (g)']
    mass_sorbitol = row['Mass Sorbitol (g)']

    mw_kno3 = 101.1032
    mw_sorbitol = 182.172

    moles_kno3 = mass_kno3 / mw_kno3
    moles_sorbitol = mass_sorbitol / mw_sorbitol
    
    T_chamber = mix.T
    gamma = mix.cp / mix.cv
    mw = mix.mean_molecular_weight
    
    R = ct.gas_constant / mw
    c_star_val = np.sqrt(gamma * R * T_chamber) / gamma

    exit_pressure = 1.01325e+06
    isp_val = (np.sqrt(2 * gamma / (gamma - 1) * R * T_chamber * 
               (1 - (exit_pressure / chamber_pressure)**((gamma - 1) / gamma)))) / 9.81
    
    chamber_temps.append(T_chamber)

df.head()

From this data we can show the relationship between $I_{sp}$ and mass of $KNO_3$ used:

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(df.iloc[:, 2], isps, marker='o')
plt.xlabel('CaCO₃ Mass (g)')
plt.ylabel('Specific Impulse (s)')
plt.title('Specific vs Mass of KNO3 used')

plt.tight_layout()
plt.show()

We can also outline the relationship between the flame temperatures and mass of $KNO_3$ used:

In [ ]:

plt.subplot(1, 2, 2)
plt.plot(df.iloc[:, 2], chamber_temps, marker='x', color='orange')
plt.xlabel('Mass of KNO3 used (g)')
plt.ylabel('Flame temperature (K)')
plt.title('Flame temperature vs Mass of KNO3 used')

plt.tight_layout()
plt.show()

And finally, the $C^*$ vs mass of $KNO_3$ used:

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(df.iloc[:, 2], c_stars, marker='x', color='red')
plt.xlabel('Mass of KNO3 used (g)')
plt.ylabel('Chamber velocity, C* (m/s)')
plt.title('C* vs Mass of KNO3 used')

plt.tight_layout()
plt.show()

## What we can learn from this data

We'll find that with more mass of $KNO_3$ used, do these critical factors i.e. $I_{sp}$, chamber temperature and $C^*$ increase. In reality, KNSB grains made with such concentrations of $KNO_3$ are harder to process and for the motor casing to handle.

We'll also find that with less mass of $KNO_3$ used, the motor is cooler but less energetic. We can see that at $65\%:35\%$ i.e. when mass of $KNO_3$ used is $944.84g$, we have the "[sweet spot](https://tenor.com/blpXi.gif)".

It's also worth mentioning that when too much oxidizer is used, the KNSB grain becomes brittle and sensitive to moisture while when too much fuel is used, incomplete combustion and lower $I_{sp}$ is noted. ([Nakka, 2025](https://www.nakka-rocketry.net/sorb.html))

## In conclusion...

We justified the optimal KNSB ratio i.e. $65\%:35\%$, as it provides a high specific impulse with efficient combustion, *indicated by $I_{sp}$ and $C^*$*, while maintaining safe thermal characteristics and physical integrity. It is a well-characterized formulation that makes it a reliable and practical choice for our SRM.

## References

1. R. Nakka, "KNSB Propellant," *Richard Nakka's Experimental Rocketry Web Site*, 2025. [Online]. Available: https://www.nakka-rocketry.net/sorb.html
2. R. Nakka, "Solid Rocket Motor Theory - Two-phase flow," *Richard Nakka's Experimental Rocketry Web Site*, 2025. [Online]. Available: https://www.nakka-rocketry.net/th_2phf.htm
3. R. Nakka, "Solid Rocket Motor Theory -- Impulse and C-star," *Richard Nakka's Experimental Rocketry Web Site*, 2025. [Online]. Available: https://www.nakka-rocketry.net/th_imp.html
4. J. Bonnie, J. Zehe, and S. Gordon, *NASA Glenn Coefficients for Calculating Thermodynamic Properties of Individual Species*, NASA/TP—2002-211556, Glenn Research Center, Cleveland, 2002.
5. T. McReary, *EXPERIMENTAL COMPOSITE PROPELLANT: An Introduction To Properties And Preparation OF Composite Propellants: Design, Construction, Testing, and Characteristics Of Small Rocket Motors.*, 1st Edition, 2020.